In [1]:
from datetime import datetime, timedelta, date
import csv
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import seaborn as sns
from fbprophet import Prophet

In [2]:
df = pd.read_csv('../Full_Monthly.csv',index_col=0)

In [3]:
df.head()

,Date,Ticker,Sector,Industry,Cash From Operations,Current Market Cap,EBITDA Margin,Free Cash Flow,Gross Margin,Interest Expense,...,Yield.10.Yr.Vol.63,Yield.20.Yr.Vol.21,Yield.20.Yr.Vol.63,Yield.30.Yr.Vol.21,Yield.30.Yr.Vol.63,Total,OAS,Shiller.PE.Ratio,VIX.Value,VIX.VXV
1,1998-07-01,BAX,"Consumer, Non-cyclical",Healthcare-Products,146.000,15118.3532,25.5814,17.000,46.0888,39.000,...,0.100107,0.067771,0.117695,0.06957,0.126968,76.83215,72.402903,38.26,24.799999,NaN
2,1998-07-01,VOD,Communications,Telecommunications,NaN,256921.1418,NaN,NaN,NaN,NaN,...,0.100107,0.067771,0.117695,0.06957,0.126968,76.83215,78.340585,38.26,24.799999,NaN
3,1998-07-01,PXD,Energy,Oil&Gas,91.369,2398.8883,45.2357,-56.929,23.5375,41.017,...,0.100107,0.067771,0.117695,0.06957,0.126968,76.83215,147.188061,38.26,24.799999,NaN
4,1998-07-01,KMI,Energy,Pipelines,453.000,36519.0472,27.3650,-10.000,34.1947,297.000,...,0.100107,0.067771,0.117695,0.06957,0.126968,76.83215,113.533557,38.26,24.799999,NaN
5,1998-07-01,AZN,"Consumer, Non-cyclical",Pharmaceuticals,311.000,82119.0000,28.0482,-203.000,69.4947,33.000,...,0.100107,0.067771,0.117695,0.06957,0.126968,76.83215,60.176544,38.26,24.799999,NaN


In [12]:
df.dtypes.head()

Date                    datetime64[ns]
Ticker                          object
Sector                          object
Industry                        object
Cash From Operations           float64
dtype: object

### 35,064 observations

In [9]:
len(df)

35064

In [10]:
df.describe()

,Cash From Operations,Current Market Cap,EBITDA Margin,Free Cash Flow,Gross Margin,Interest Expense,Long Term Debt,Net Debt,Net Income/Net Profit (Losses),Revenue,...,Yield.10.Yr.Vol.63,Yield.20.Yr.Vol.21,Yield.20.Yr.Vol.63,Yield.30.Yr.Vol.21,Yield.30.Yr.Vol.63,Total,OAS,Shiller.PE.Ratio,VIX.Value,VIX.VXV
count,34416.000000,34647.000000,34290.000000,34290.000000,32697.000000,34389.000000,3.465000e+04,34650.000000,34416.000000,3.465000e+04,...,35064.000000,35064.000000,35064.000000,29020.000000,29282.000000,35064.000000,34999.000000,35064.000000,35064.000000,29816.000000
mean,1280.759140,50383.998924,22.652417,652.872743,41.294735,124.105400,1.109283e+04,6095.889031,669.087025,9.499222e+03,...,0.165653,0.091606,0.154919,0.087695,0.148144,153.386671,138.259805,26.363448,19.832317,0.937338
std,6088.493883,66479.485752,15.929808,4815.461803,22.631193,315.221063,3.131732e+04,24090.633511,2022.226901,4.921038e+04,...,0.088155,0.055520,0.080396,0.061409,0.085300,65.231383,103.342547,6.091675,8.051695,0.196559
min,-220042.935500,398.809866,-285.937768,-300994.398610,-369.349226,-62.463218,-3.877647e+02,-200675.000000,-93846.225698,-6.145949e+01,...,0.048840,0.030150,0.061374,0.027001,0.034493,76.832150,-1503.125000,13.320000,9.510000,0.599930
25%,263.288751,13244.955420,12.957125,51.967321,24.965483,29.173929,2.289970e+03,839.190577,139.286215,2.187848e+03,...,0.103352,0.063520,0.103344,0.059825,0.095271,112.972668,84.694456,22.600000,13.840000,0.823625
50%,602.837257,25492.565064,20.772672,282.712999,39.885892,72.580527,5.577126e+03,3522.240696,329.052014,3.971007e+03,...,0.137183,0.080838,0.134954,0.075141,0.125602,136.313962,114.509722,25.690000,17.750000,0.900752
75%,1286.654551,59057.008908,31.180740,788.148975,58.133310,146.952248,1.136533e+04,9001.706545,765.715734,8.725784e+03,...,0.203721,0.101644,0.180300,0.096380,0.169973,173.348797,159.228902,27.550000,24.059999,1.017011
max,450738.000000,796064.939400,165.589996,377795.000000,159.437267,12089.706719,1.052778e+06,879685.091457,73979.088458,2.279306e+06,...,0.671005,0.585437,0.627464,0.644613,0.636794,546.125830,2459.297562,44.190000,59.889999,1.915768


In [50]:
df.Ticker.value_counts()

AZN      234
CAT      234
BRK      234
DUK      234
DVN      234
SRE      234
BA       234
APC      234
MRK      234
RTN      234
PFE      234
VLO      234
CTL      234
JCI      234
JNJ      234
KR       234
CMCSA    234
MMM      234
PPL      234
VOD      234
EMR      234
WEC      234
EXC      234
HSY      234
ROK      234
LMT      234
UPS      234
MCK      234
LLY      234
PX       234
        ... 
LRCX      33
ZBH       30
QCOM      30
ROST      30
GS        27
KHC       27
CLR       27
ADP       27
V         24
SNE       24
HPE       24
ETE       21
EA        21
SPGI      18
FTV       18
CHTR      18
JD        18
ATVI      15
MT        15
NVDA      15
STZ       12
IMO       12
AVGO       9
DLTR       9
DXC        9
MU         6
VMW        3
CXO        3
ANDV       3
PSA        3
Name: Ticker, Length: 239, dtype: int64

### Range of values for OAS is from -1500 to 2500

In [8]:
df['Date'] = pd.to_datetime(df['Date'])

In [13]:
df_aapl = df[df.Ticker == 'AAPL']
oas_max = df_aapl.OAS.max()

In [12]:
df_aapl.head()

,Date,Ticker,Sector,Industry,Cash From Operations,Current Market Cap,EBITDA Margin,Free Cash Flow,Gross Margin,Interest Expense,...,Yield.10.Yr.Vol.63,Yield.20.Yr.Vol.21,Yield.20.Yr.Vol.63,Yield.30.Yr.Vol.21,Yield.30.Yr.Vol.63,Total,OAS,Shiller.PE.Ratio,VIX.Value,VIX.VXV
24514,2013-07-01,AAPL,Technology,Computers,7828.000000,372202.482700,30.843900,5943.000000,36.871200,53.000000,...,0.262504,0.139378,0.250112,0.120690,0.223990,149.030998,74.412388,23.49,13.45,0.750558
24694,2013-08-01,AAPL,Technology,Computers,7185.139726,388251.672477,30.868757,5295.067615,36.872520,64.782264,...,0.297837,0.057719,0.262457,0.057871,0.225910,135.772500,67.831581,23.36,17.01,1.169072
24871,2013-09-01,AAPL,Technology,Computers,7443.704916,406703.896894,31.026814,5536.601103,36.905112,75.078502,...,0.211542,0.093770,0.192093,0.082531,0.167543,139.048144,75.077356,23.44,16.60,0.919668
25126,2013-10-01,AAPL,Technology,Computers,9505.121032,429962.190593,31.451269,7557.066338,37.000245,82.402688,...,0.134206,0.091167,0.123322,0.078303,0.105446,142.691810,81.054695,23.83,13.75,0.852449
25326,2013-11-01,AAPL,Technology,Computers,13893.734584,459021.871708,32.225477,11877.972705,37.185124,85.632205,...,0.127044,0.062301,0.103530,0.057226,0.086108,133.898188,80.189594,24.64,13.70,0.909695


In [18]:
df_aapl['OAS_pct_of_max'] = df_aapl['OAS']/oas_max
df_aapl.head()

/Users/veronicahsieh/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Date,Ticker,Sector,Industry,Cash From Operations,Current Market Cap,EBITDA Margin,Free Cash Flow,Gross Margin,Interest Expense,...,Yield.20.Yr.Vol.21,Yield.20.Yr.Vol.63,Yield.30.Yr.Vol.21,Yield.30.Yr.Vol.63,Total,OAS,Shiller.PE.Ratio,VIX.Value,VIX.VXV,OAS_pct_of_max
24514,2013-07-01,AAPL,Technology,Computers,7828.000000,372202.482700,30.843900,5943.000000,36.871200,53.000000,...,0.139378,0.250112,0.120690,0.223990,149.030998,74.412388,23.49,13.45,0.750558,0.742723
24694,2013-08-01,AAPL,Technology,Computers,7185.139726,388251.672477,30.868757,5295.067615,36.872520,64.782264,...,0.057719,0.262457,0.057871,0.225910,135.772500,67.831581,23.36,17.01,1.169072,0.677039
24871,2013-09-01,AAPL,Technology,Computers,7443.704916,406703.896894,31.026814,5536.601103,36.905112,75.078502,...,0.093770,0.192093,0.082531,0.167543,139.048144,75.077356,23.44,16.60,0.919668,0.749360
25126,2013-10-01,AAPL,Technology,Computers,9505.121032,429962.190593,31.451269,7557.066338,37.000245,82.402688,...,0.091167,0.123322,0.078303,0.105446,142.691810,81.054695,23.83,13.75,0.852449,0.809021
25326,2013-11-01,AAPL,Technology,Computers,13893.734584,459021.871708,32.225477,11877.972705,37.185124,85.632205,...,0.062301,0.103530,0.057226,0.086108,133.898188,80.189594,24.64,13.70,0.909695,0.800386


In [22]:
df_aapl['oas_rank'] = pd.cut(df_aapl.OAS_pct_of_max, bins = 4,labels = [1,2,3,4])
df_aapl.head(15)

/Users/veronicahsieh/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Date,Ticker,Sector,Industry,Cash From Operations,Current Market Cap,EBITDA Margin,Free Cash Flow,Gross Margin,Interest Expense,...,Yield.20.Yr.Vol.63,Yield.30.Yr.Vol.21,Yield.30.Yr.Vol.63,Total,OAS,Shiller.PE.Ratio,VIX.Value,VIX.VXV,OAS_pct_of_max,oas_rank
24514,2013-07-01,AAPL,Technology,Computers,7828.000000,372202.482700,30.843900,5943.000000,36.871200,53.000000,...,0.250112,0.120690,0.223990,149.030998,74.412388,23.49,13.450000,0.750558,0.742723,2
24694,2013-08-01,AAPL,Technology,Computers,7185.139726,388251.672477,30.868757,5295.067615,36.872520,64.782264,...,0.262457,0.057871,0.225910,135.772500,67.831581,23.36,17.010000,1.169072,0.677039,2
24871,2013-09-01,AAPL,Technology,Computers,7443.704916,406703.896894,31.026814,5536.601103,36.905112,75.078502,...,0.192093,0.082531,0.167543,139.048144,75.077356,23.44,16.600000,0.919668,0.749360,2
25126,2013-10-01,AAPL,Technology,Computers,9505.121032,429962.190593,31.451269,7557.066338,37.000245,82.402688,...,0.123322,0.078303,0.105446,142.691810,81.054695,23.83,13.750000,0.852449,0.809021,3
25326,2013-11-01,AAPL,Technology,Computers,13893.734584,459021.871708,32.225477,11877.972705,37.185124,85.632205,...,0.103530,0.057226,0.086108,133.898188,80.189594,24.64,13.700000,0.909695,0.800386,3
25516,2013-12-01,AAPL,Technology,Computers,18981.134366,486841.552695,33.148236,16920.152362,37.463737,85.719144,...,0.096095,0.075464,0.084736,128.380481,74.119777,24.86,13.720000,0.903226,0.739802,2
25720,2014-01-01,AAPL,Technology,Computers,22376.148279,503532.296208,33.917515,20360.129627,37.831841,84.350704,...,0.116930,0.035200,0.102891,115.826165,64.386152,24.86,18.410000,1.246445,0.642649,1
25912,2014-02-01,AAPL,Technology,Computers,22030.203408,501109.839741,34.267630,20201.456787,38.280083,83.124599,...,0.099058,0.104111,0.092919,118.645076,72.042631,24.59,14.000000,0.695134,0.719070,2
26139,2014-03-01,AAPL,Technology,Computers,18742.107155,487406.356023,34.235126,17165.809793,38.756777,82.889622,...,0.119902,0.048216,0.113709,111.687369,66.965142,24.96,13.880000,0.820331,0.668391,2
26312,2014-04-01,AAPL,Technology,Computers,14718.566096,478074.520711,34.005991,13318.852279,39.189310,84.124261,...,0.115981,0.059909,0.105833,107.469752,54.189676,24.79,13.410000,0.931250,0.540876,1


In [26]:
df_aapl.oas_rank.value_counts()

2    18
1    18
3    12
4     6
Name: oas_rank, dtype: int64

In [31]:
df_aapl['outlier'] = np.where((df_aapl.oas_rank == 1)|(df_aapl.oas_rank == 4),1,0)
df_aapl.head()

/Users/veronicahsieh/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Date,Ticker,Sector,Industry,Cash From Operations,Current Market Cap,EBITDA Margin,Free Cash Flow,Gross Margin,Interest Expense,...,Yield.30.Yr.Vol.21,Yield.30.Yr.Vol.63,Total,OAS,Shiller.PE.Ratio,VIX.Value,VIX.VXV,OAS_pct_of_max,oas_rank,outlier
24514,2013-07-01,AAPL,Technology,Computers,7828.000000,372202.482700,30.843900,5943.000000,36.871200,53.000000,...,0.120690,0.223990,149.030998,74.412388,23.49,13.45,0.750558,0.742723,2,0
24694,2013-08-01,AAPL,Technology,Computers,7185.139726,388251.672477,30.868757,5295.067615,36.872520,64.782264,...,0.057871,0.225910,135.772500,67.831581,23.36,17.01,1.169072,0.677039,2,0
24871,2013-09-01,AAPL,Technology,Computers,7443.704916,406703.896894,31.026814,5536.601103,36.905112,75.078502,...,0.082531,0.167543,139.048144,75.077356,23.44,16.60,0.919668,0.749360,2,0
25126,2013-10-01,AAPL,Technology,Computers,9505.121032,429962.190593,31.451269,7557.066338,37.000245,82.402688,...,0.078303,0.105446,142.691810,81.054695,23.83,13.75,0.852449,0.809021,3,0
25326,2013-11-01,AAPL,Technology,Computers,13893.734584,459021.871708,32.225477,11877.972705,37.185124,85.632205,...,0.057226,0.086108,133.898188,80.189594,24.64,13.70,0.909695,0.800386,3,0


In [47]:
features = ['Current Market Cap','Shares.Outstanding','Gross Margin','Interest Expense','Cash From Operations','EBITDA Margin','Net Debt','Total Current Liabilities','Total Assets','Revenue','Long Term Debt','Stock.Price','Net Income/Net Profit (Losses)','Short and Long Term Debt','Free Cash Flow']

In [48]:
df_aapl_features = df_aapl[features]
df_aapl_y = df_aapl.outlier
df_aapl_features.head()

,Current Market Cap,Shares.Outstanding,Gross Margin,Interest Expense,Cash From Operations,EBITDA Margin,Net Debt,Total Current Liabilities,Total Assets,Revenue,Long Term Debt,Stock.Price,Net Income/Net Profit (Losses),Short and Long Term Debt,Free Cash Flow
24514,372202.482700,6570.543,36.871200,53.000000,7828.000000,30.843900,-129662.000000,36319.000000,199856.000000,35323.000000,16958.000000,64.647141,6900.000000,16958.000000,5943.000000
24694,388251.672477,6359.479,36.872520,64.782264,7185.139726,30.868757,-128286.097452,37977.924284,200303.734317,33945.931688,16872.989904,69.602859,6538.685473,16868.090994,5295.067615
24871,406703.896894,6359.479,36.905112,75.078502,7443.704916,31.026814,-127887.625556,40115.139841,202020.135547,33990.722870,16846.916270,68.107140,6561.064145,16840.489367,5536.601103
25126,429962.190593,6359.479,37.000245,82.402688,9505.121032,31.451269,-129444.014967,43208.937945,206273.870600,36879.233040,16938.715559,74.671425,7350.829215,16937.502496,7557.066338
25326,459021.871708,6298.166,37.185124,85.632205,13893.734584,32.225477,-133531.946568,47471.886718,213712.365587,43464.013960,17166.814987,79.438568,9140.264472,17178.612851,11877.972705


In [38]:
from sklearn.linear_model import LogisticRegression

In [49]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(df_aapl_features, df_aapl_y)